# Flow Free Description

Flow Free is a iOS and Android puzzle game that resembles another game called Numberlink. Each puzzle has a grid with colored dots in some spaces of the grid. The objective of the game is to connect dots of the same color by drawing 'pipes' between them. Every square on the grid should be occupied by a color or a pipe, and no pipes can intersect each other. The difficulty increases as the dimensions of the grid increase.

# Flow Free Solver

This solver uses an A-star search algorithm in order to find the path to the goal node, which is the solved Flow Free puzzle.

## PuzzleNode class
The PuzzleNode class points to the parent node, the current state, the f-cost (which is the combination of the g-cost and the heuristic costs), the g-cost, and indicates whether the node has been pruned or not. \medskip

\no indent The state is represented by a list of lists. For each color, there is a beginning, represented by a capital letter, and an end point, represented by the letter and '_end' attached to it. \medskip

\noindent Within the class, there is a function that selects the color whose two ends are closest to each other to move. This is because it is generally a good strategy to connect the nodes closest together first, because the path is very straightforward and there are likely not many variations of how they would be connected. Considering this is important since every time a color is connected, the moves that can be taken are constrained—if a move early on turns out to be incorrect, much time might have been wasted on expanding an incorrect node.

## Heuristic: Manhattan
The first heuristic used is the Manhattan distance, which essentially shows how many steps away a color's tail is from its end. This heuristic adds up all of the distances for every color.

## Heuristic: Misplaced Squares
This heuristic compares the current state and the goal state and counts the number of squares that are incorrectly filled in. (Lower case and upper case versions of the same letter are assumed to be the same value.)

## Heuristic: Trapped Colors
This heuristic looks at the number of trapped colors there are on the board. A trapped color is one that is surrounded by other colors/walls on all four sides. When this happens, the color cannot move and thus is a state in which the solution cannot be derived.

## Solver
The solver implements A-star search to get to the correct solution. In the actual Flow Free game, the pipe is essentially a trail and only extends from the current end point of the trail. To represent this, every time a color is extended a square, the extended square is the color's capital letter, and the previous square becomes the color's lower case letter. Thus, in each expansion of a node, the solver is looking to extend the trail so that the capital letter is next to its end counterpart. (An example trail: a,a,a,a,A,A_end.)

\noindent It takes as input:
- n - the dimension of the puzzle
- state - the starting board state
- cur_heur - the heuristic(s) that will be used
- goal - the goal state (manually defined)
- colors - a list of the colors in the game

\noindent The function outputs:
- the board dimension
- the frontier size
- the number of steps taken to get to the goal state


In [5]:
from queue import PriorityQueue
from copy import deepcopy

def find_index(lst, char):
    for sub_list in lst:
        if char in sub_list:
            return lst.index(sub_list), sub_list.index(char)

poss_directions = [(-1,0),(1,0),(0,-1),(0,1)]

class PuzzleNode:
    def __init__(self, state, fval, gval, colors, parent=None):
        self.parent = parent
        self.state = state
        self.fval = fval # gval + hval
        self.gval = gval # cost to node
        self.pruned = False # not pruned yet
        self.colors = colors
        
    def __str__(self):
        return "\n".join([" ".join([str(x) for x in row]) 
                          for row in self.state])
    
    # Compare fvals to order in queue by cost
    def __lt__(self, other):
        return self.fval < other.fval
        
        
    # get position of color that is closest to each other and not connected yet
    def select_closest(self):
        not_connected = {}

        for color in self.colors:
            # find start point of the color (capital letter)
            row, col = find_index(self.state, color)
            n = len(self.state)
    
            count = 0
            goal_row, goal_col = find_index(self.state, color+'_end')
            
            #check if color even needs to be connected
            connected = 0
            for add_row, add_col in poss_directions:
                if row+add_row in range(0,n) and col+add_col in range(0,n):
                    if self.state[row+add_row][col+add_col] == color:
                        connected +=1
            
            # if the color isn't already connected
            if connected == 0:
                # get the manhattan distance and add to dictionary
                count += abs(row - goal_row) + abs(col - goal_col)-1
                not_connected[color] = count
        
        #get the color that has lowest manhattan distance

        no_zeros = {k: v for k, v in not_connected.items() if v > 0}
        color_to_pick = min(no_zeros, key=lambda k: not_connected[k])
        
        return color_to_pick

# goal is only used by the misplaced heuristic but is included as input for 
# all for standardization purposes
def heur_manhattan(state, goal, colors):
    # get manhattan distance from a specific color to its counterpart
    dist = 0
    n = len(state)
    for color in colors:
        row,col = find_index(state, color)
        
        # find the end color
        goal_row, goal_col = find_index(state, color+'_end')
            
        # default not connected
        connected = False
        
        # for the surrounding squares
        for add_row, add_col in poss_directions:
            if row+add_row in range(0,n) and col+add_col in range(0,n):
                # if the tail of the trail is next to the end, then they 
                # are connected
                if state[row+add_row][col+add_col] == color:
                    connected = True
            
        # if the color isn't already connected
        if not connected:
            # get the manhattan distance and add to dictionary
            dist += abs(row - goal_row) + abs(col - goal_col) -1
    
    return dist

def heur_misplaced(state, goal_state, colors):
    cost = 0
    for row in state:
        for elem in row:
            x,y = state.index(row), row.index(elem)
            if goal_state[x][y] != elem or goal_state[x][y] != elem.lower():
                cost += 1
    return cost
    
def neighbor_color(state, element_position):
    neighbors = []
    n = len(state)
    row,col = element_position
    for add_row, add_col in poss_directions:
        if row+add_row in range(0,n) and col+add_col in range(0,n):
            neighbors.append(state[row+add_row][col+add_col])
    return neighbors

def heur_trapped(state, goal, colors):
    cost = 0
    for color in colors:
        out = [(ind,ind2) for ind,i in enumerate(state) 
                  for ind2,y in enumerate(i) if (y == color 
                                                 or y == color.lower() 
                                                 or y== color+'_end')]
        # for each instance of the color
        for elem in out:
            neighbors = neighbor_color(state,elem)
            # if theres no open space and its own color is not in neighbors
            if None not in neighbors:
                if (color not in neighbors 
                     or color.lower() not in neighbors 
                     or color+'_end' not in neighbors):
                    cost += 1
    return cost

def solve_flow(n, state, cur_heur, goal, colors):
    
    start_node = PuzzleNode(state, cur_heur(state, goal, colors), 0, colors)
    
    goal_state = goal
    
    costs_db = {str(state): start_node}
    
    frontier = PriorityQueue()
    frontier.put(start_node)
    
    expansion_counter = 0
    
    while not frontier.empty():
        
        cur_node = frontier.get()
        
        if cur_node.pruned:
            continue
        
        if cur_node.state == goal_state: 
            break
        
        # get the color that is easiest to connect
        cur_color = cur_node.select_closest()

        # get index of that color
        color_row, color_col = find_index(cur_node.state, cur_color)

        
        for add_row, add_col in poss_directions:
            
            col_move = color_col + add_col
            row_move = color_row + add_row
            
            # if its in the board and its empty, then continue 
            # (if not, you can't do anything anyways)
            if (col_move in range(0,n) 
                and row_move in range(0,n) 
                and cur_node.state[row_move][col_move] is None):
                child_state = deepcopy(cur_node.state)
                
                # extend the end of the line of the color
                child_state[row_move][col_move] = cur_color
                # previous position is now lowercase
                child_state[color_row][color_col] = cur_color.lower()
                
                gval = cur_node.gval + 1
                
                # If the child node is already in the cost database
                # (i.e. explored)
                if str(child_state) in costs_db:
                    if costs_db[str(child_state)].gval > gval:
                        # Mark existing value for deletion from frontier
                        costs_db[str(child_state)].pruned = True 
                    else:
                        # ignore this child, 
                        # since a better path has already been found previously.
                        continue 
                
                # Heuristic cost from next node to goal
                hval = cur_heur(child_state, goal,colors)
                
                # create new child node
                next_node = PuzzleNode(child_state, gval+hval, 
                                       gval, colors, cur_node) 
                frontier.put(next_node)
                costs_db[str(child_state)] = next_node #Mark the node as explored
    
        expansion_counter += 1

    print ("Board size:", len(state),'x',len(state))
    print ("Frontier size: ", frontier.qsize())
    print ("Number of expansions (steps): ", expansion_counter)
    # print (cur_node.__str__())
            
    return frontier.qsize(), expansion_counter    

# Analysis

## Test Results
The solver was run with three puzzles of varying dimensions (5x5, 6x6, 7x7). Each puzzle was run with each of the three heuristic functions. The tests (below) show that the misplaced heuristic performed far better than all the others and significatnly reduced the number of steps taken. The worst performing heuristic was the trapped heuristic. This is likely due to the fact that it is not really useful for reducing the explored states, particularly in the beginning, when there are not many squares filled in to begin with. It assesses all states as essentially the same unless there is a blocked node.

Because of extremely long running time, the manhattan and trapped heuristics were not tested with larger boards. The misplaced heuristic performed well with even the larger 8x8 and 9x9 boards.



All test puzzles are replications of actual levels in Flow Free. Flow Free has puzzles from dimensions of 5 to 9.

## Possible Improvements
### Improvements in Performance
Some of the heuristics did not perform well. Improvements could be made to the heuristics to run more efficiently. Alternatively, different heuristics could be made to better guide the search.

Another improvement to performance could be to have a different criteria in selecting a color to extend. Currently, it chooses the color whose trail and end point are closest together. An alternative could be to choose the most constrained trail/point to extend. This might be an improvement because a constrained point (i.e. is surrounded on three sides) can only go in one direction anyways, so it is likely to be a correct move.
### Improvements in experience
An improvement to the overall experience of using this solver would be to create a way so that the goal state does not have to be manually inputted each time. It somewhat defeats the purpose of using the algorithm, since it requires the answer is already known. Instead of inputting the goal manually, the algorithm can check each state to see if it satisfies all the requirements of a goal state (i.e. there is one start and one end for all color trails, no empty squares, etc).

Another improvement would be to implement a class for the pipe/trail. Right now, the board has to be represented by a separate character for the trail, the end of the trail, and the end point that needs to connect to the trail. By making a class, the start and ends could be more easily accessed and the board could be represented in a more simple way.

In [3]:
board_5x5 = [[None,'A','B','C',None],
         [None,None,None,'D',None],
         [None,None,'D_end',None,None],
         ['A_end',None,None,'E',None],
         ['B_end',None,'E_end','C_end',None]]


board_5x5_goal = [['a','a','b','c','c'],
             ['a','b','b','d','c'],
             ['A','b','D_end','D','c'],
             ['A_end','b','E','e','c'],
             ['B_end','B','E_end','C_end','C']]

board_5x5_colors = ['A','B','C','D','E']

board_6x6 = [[None,None,None,None,None,'O'],
             [None,'G',None,None,None,'Y'],
             [None,'R',None,'R_end',None,None],
             [None,None,None,None,None,None],
             [None,None,'B',None,'B_end',None],
             ['O_end',None,'G_end','Y_end',None,None]]

board_6x6_goal = [['o','o','o','o','o','o'],
             ['o','g','g','g','g','y'],
             ['o','r','R','R_end','g','y'],
             ['o','g','g','g','g','y'],
             ['O','g','b','B','B_end','y'],
             ['O_end','G','G_end','Y_end','Y','y']]

board_6x6_colors = ['O','G','B','Y','R']

board_7x7 = [[None,None,None,None,None,None,None],
             [None,'G',None,None,None,'Y',None],
             [None,None,'Y_end',None,None,'B','R'],
             [None,None,None,None,None,None,'P'],
             [None,None,None,None,None,'O','T'],
             ['P_end','R_end','G_end','B_end',None,None,None],
             [None,None,None,None,None,'O_end','T_end']]

board_7x7_goal = [['r','r','r','r','r','r','r'],
             ['r','g','Y','y','y','y','r'],
             ['r','g','Y_end','b','b','b','r'],
             ['r','g','g','b','p','p','p'],
             ['r','R','G','B','p','o','t'],
             ['P_end','R_end','G_end','B_end','p','O','T'],
             ['P','p','p','p','p','O_end','T_end']]
             
board_7x7_colors = ['G','P','B','O','R','Y','T']

# Larger boards
board_8x8 = [['G',None,None,None,None,None,'P','G_end'],
             [None,None,None,None,'O',None,None,None],
             [None,None,None,None,'T',None,None,None],
             [None,None,'R',None,'B',None,None,None],
             ['O_end',None,None,None,'Y',None,None,None],
             ['Y_end',None,'R_end',None,None,None,None,None],
             [None,None,'T_end','B_end',None,None,'P_end',None],
             [None,None,None,None,None,None,None,None]]

board_8x8_goal = [['g','g','g','g','g','g','p','G_end'],
             ['o','o','o','o','o','g','p','G'],
             ['o','t','t','t','t','g','p','g'],
             ['O','t','r','b','b','g','p','g'],
             ['O_end','t','R','b','y','g','p','g'],
             ['Y_end','t','R_end','B','y','g','P','g'],
             ['Y','T','T_end','B_end','y','g','P_end','g'],
             ['y','y','y','y','y','g','g','g']]

board_8x8_colors = ['G','P','O','T','B','R','Y']

board_9x9 = [[None,None,None,None,None,None,None,None,'T'],
             [None,None,None,None,'N',None,'R',None,None],
             [None,None,None,None,None,None,'P',None,None],
             [None,None,None,'P_end',None,None,None,None,None],
             [None,None,None,'B','G','Y',None,None,None],
             [None,None,None,None,None,'M',None,'M_end',None],
             [None,None,None,None,None,'O',None,None,None],
             [None,'N_end','R_end','B_end',None,None,'G_end',None,None],
             [None,None,None,None,None,None,'Y_end','O_end','T_end']]

board_9x9_goal = [['y','y','y','y','y','y','y','y','t'],
             ['y','n','n','n','n','r','r','y','t'],
             ['y','n','r','r','r','r','p','y','t'],
             ['y','n','r','P_end','P','p','p','y','t'],
             ['y','n','r','b','g','y','y','y','t'],
             ['y','n','r','b','g','m','M','M_end','t'],
             ['y','N','R','B','g','o','o','o','t'],
             ['y','N_end','R_end','B_end','g','G','G_end','O','T'],
             ['y','y','y','y','y','Y','Y_end','O_end','T_end']]

board_9x9_colors = ['P','M','N','R','B','G','Y','O','T']

print ("Manhattan heuristic test:")
solve_flow(len(board_5x5), board_5x5, heur_manhattan, 
           board_5x5_goal, board_5x5_colors)
solve_flow(len(board_6x6), board_6x6, heur_manhattan, 
           board_6x6_goal, board_6x6_colors)
solve_flow(len(board_7x7), board_7x7, heur_manhattan, 
           board_7x7_goal, board_7x7_colors)

print ("Misplaced heuristic test")
solve_flow(len(board_5x5), board_5x5, heur_misplaced, 
           board_5x5_goal, board_5x5_colors)
solve_flow(len(board_6x6), board_6x6, heur_misplaced, 
           board_6x6_goal, board_6x6_colors)
solve_flow(len(board_7x7), board_7x7, heur_misplaced, 
           board_7x7_goal, board_7x7_colors)
solve_flow(len(board_8x8), board_8x8, heur_misplaced, 
           board_8x8_goal, board_8x8_colors)
solve_flow(len(board_9x9), board_9x9, heur_misplaced, 
           board_9x9_goal, board_9x9_colors)

print ("Trapped heuristic test")
solve_flow(len(board_5x5), board_5x5, heur_trapped, 
           board_5x5_goal, board_5x5_colors)
solve_flow(len(board_6x6), board_6x6, heur_trapped, 
           board_6x6_goal, board_6x6_colors)
solve_flow(len(board_7x7), board_7x7, heur_trapped, 
           board_7x7_goal, board_7x7_colors)

Manhattan heuristic test:
Board size: 5 x 5
Frontier size:  19
Number of expansions (steps):  103


Board size: 6 x 6
Frontier size:  479
Number of expansions (steps):  1426


Board size: 7 x 7
Frontier size:  7524
Number of expansions (steps):  32164
Misplaced heuristic test
Board size: 5 x 5
Frontier size:  26
Number of expansions (steps):  56
Board size: 6 x 6
Frontier size:  38
Number of expansions (steps):  55
Board size: 7 x 7
Frontier size:  254
Number of expansions (steps):  331


Board size: 8 x 8
Frontier size:  1206
Number of expansions (steps):  1510
Board size: 9 x 9
Frontier size:  252
Number of expansions (steps):  432
Trapped heuristic test


Board size: 5 x 5
Frontier size:  0
Number of expansions (steps):  328


Board size: 6 x 6
Frontier size:  6
Number of expansions (steps):  16952


Board size: 7 x 7
Frontier size:  1
Number of expansions (steps):  319310


(1, 319310)

# References
- The original Flow Free game (available on App Store and Play Store)
- Referenced code from my A* search assignment (CS152 Assignment 1)

# Appendix

## LOs to be graded:
Note: this project idea was discussed over the Forum, so there is no written proposal for this particular idea, but the indended LOs to be graded are as follows:
- #aicoding
- #search
